<a href="https://colab.research.google.com/github/NandakishorV/Imageprocessing-Fungi/blob/main/1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **IMPORTING LIBRARIES**

In [1]:
!pip install fastai

In [2]:
from fastai import *
from fastai.vision import *
from fastai.metrics import error_rate
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

In [3]:
import numpy as np #for numerical computations
import pandas as pd #for dataframe operations

from matplotlib import pyplot as plt #for viewing images and plots
%matplotlib inline 
#So that Matplotlib plots don't open in separate windows outside the notebook

import urllib #For fetching data from Web URLs

import cv2   #For image processing

from sklearn.preprocessing import LabelEncoder    #For encoding categorical variables
from sklearn.model_selection import train_test_split #For splitting of dataset

#All tensorflow utilities for creating, training and working with a CNN
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
import os

# **IMPORTING DATASET**

In [12]:
rm -r '../content/sample_data'

In [4]:
 ! pip install -q kaggle

In [5]:
from google.colab import files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nandakishorv","key":"1a4647cafd9e13f75fa3ca7efd3daff2"}'}

In [13]:
! mkdir ~.kaggle

mkdir: cannot create directory ‘~.kaggle’: File exists


In [14]:
! cp kaggle.json ~/.kaggle/

In [15]:
! chmod 600 ~/.kaggle/kaggle.json

In [16]:
! kaggle competitions download -c fungiclef2022

100% 8.72G/8.73G [01:12<00:00, 138MB/s]
100% 8.73G/8.73G [01:12<00:00, 130MB/s]


In [17]:
! unzip fungiclef2022.zip

Streaming output truncated to the last 5000 lines.
  inflating: DF21-images-300/DF21_300/4-3052905332.JPG  
  inflating: DF21-images-300/DF21_300/4-3052906308.JPG  
  inflating: DF21-images-300/DF21_300/4-3052906323.JPG  
  inflating: DF21-images-300/DF21_300/4-3052907309.JPG  
  inflating: DF21-images-300/DF21_300/4-3052907312.JPG  
  inflating: DF21-images-300/DF21_300/4-3052908303.JPG  
  inflating: DF21-images-300/DF21_300/4-3052910309.JPG  
  inflating: DF21-images-300/DF21_300/4-3052910342.JPG  
  inflating: DF21-images-300/DF21_300/4-3052910345.JPG  
  inflating: DF21-images-300/DF21_300/4-3052911303.JPG  
  inflating: DF21-images-300/DF21_300/4-3052911327.JPG  
  inflating: DF21-images-300/DF21_300/4-3052911352.JPG  
  inflating: DF21-images-300/DF21_300/4-3052912326.JPG  
  inflating: DF21-images-300/DF21_300/4-3052912336.JPG  
  inflating: DF21-images-300/DF21_300/4-3052913304.JPG  
  inflating: DF21-images-300/DF21_300/4-3052913329.JPG  
  inflating: DF21-images-300/DF21_300

# **IMAGE SEGREGATION**

***Reading train_CSV*** 

In [19]:
df = pd.read_csv(r'../content/DF20-train_metadata.csv')
df.head()

,gbifID,eventDate,year,month,day,countryCode,locality,taxonID,scientificName,kingdom,...,level2Name,ImageUniqueID,Substrate,rightsHolder,Latitude,Longitude,CoorUncert,Habitat,image_path,class_id
0,2238546328,2018-04-16T00:00:00,2018.0,4.0,16.0,DK,Ulvedal Plantage,30872.0,Ramalina farinacea (L.) Ach.,Fungi,...,Viborg,2238546328-30620,bark of living trees,Ulfva Melchior Hvidegaard,56.299706,9.258110,50.0,Mixed woodland (with coniferous and deciduous ...,2238546328-30620.JPG,1273
1,2558871973,2020-01-03T00:00:00,2020.0,1.0,3.0,DK,Slotshegn,15256.0,Hysterium acuminatum Fr.,Fungi,...,Frederikssund,2558871973-53941,dead wood (including bark),Ole Martin,55.861899,11.975973,50.0,Deciduous woodland,2558871973-53941.JPG,708
2,2238503501,2017-08-22T00:00:00,2017.0,8.0,22.0,DK,Petersborg Strandenge,61200.0,Gliophorus perplexus (A.H.Sm. & Hesler) Kovalenko,Fungi,...,Vesthimmerland,2238503501-245559,soil,Per Taudal Poulsen,56.975158,9.285525,75.0,natural grassland,2238503501-245559.JPG,535
3,2446759075,2019-10-26T00:00:00,2019.0,10.0,26.0,DK,Klintebjerg,30530.0,Lecidella scabra (Taylor) Hertel & Leuckert,Fungi,...,Odsherred,2446759075-197643,stone,Susanne Rabenborg,55.960242,11.583103,15.0,gravel or clay pit,2446759075-197643.JPG,832
4,2238472345,2016-08-21T00:00:00,2016.0,8.0,21.0,DK,Blåbjerg,63728.0,"Russula fragilis Fr., 1838",Fungi,...,Varde,2238472345-167057,soil,Tom Smidth,55.742985,8.250188,50.0,Mixed woodland (with coniferous and deciduous ...,2238472345-167057.JPG,1338


***Function for Image_Enhancement***

-Optional

In [20]:
def imgopen(img_name):
    img = cv2.imread('../content/DF20-300px/DF20_300/'+img_name)
    img=cv2.resize(img,(150,150))
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    enhanced = cv2.equalizeHist(gray)
    return enhanced

***Saving folders in drive***

-Run once

In [21]:
os.mkdir('../content/drive/MyDrive/train')

***Creating 1604 folders with class_id as name***

-Run once

In [24]:
c=0
for d in df['class_id']:
  path='../content/drive/MyDrive/train/'+str(d)
  if not os.path.exists(path):
    os.mkdir(path)
    c+=1
print (c)

1604


***Moving images into respective class_id folders***

-Run once

In [25]:
for index, row in df.iterrows():
  cv2.imwrite('../content/drive/MyDrive/train/'+str(row[33])+'/'+row[32].lower(),imgopen(row[32].lower()))